Import Libraries and Dependencies

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

Format Input (Change from .txt to .csv)

In [2]:
def change(name):
    f = open(f"{name}.txt", "r")

    out = open(f"{name}.csv", "w")

    for line in f:
        change = line.replace("\t", ",")
        out.write(change)


In [3]:
# change("FacilityMultiple")
# change("Demand")
# change("TransCost")

Get Data from csv files

In [4]:
def read_file(filename):
    with open(filename,"r") as f:
        df = pd.read_csv(f,header=None)
        return df

In [5]:
shops = read_file("Demand.csv").to_numpy()
transport = read_file("TransCost.csv").to_numpy()

In [6]:
transport_extra = []
for arr in transport:
    for i in range(3):
        transport_extra.append(arr)
transport_extra = np.array(transport_extra)
transport_extra.shape

(30, 200)

In [7]:
temp = read_file("FacilityMultiple.csv")
facilities = np.empty((10, 3, 4))

for i in range(10):
    facilities[i] = temp.groupby(0).get_group(i + 1).iloc[:, 1:].reset_index(drop=True).to_numpy()

facilities

array([[[5.21e+00, 8.07e+00, 1.00e+03, 6.80e+02],
        [5.21e+00, 8.07e+00, 1.20e+03, 7.80e+02],
        [5.21e+00, 8.07e+00, 1.50e+03, 9.20e+02]],

       [[4.40e+00, 7.31e+00, 1.00e+03, 7.00e+02],
        [4.40e+00, 7.31e+00, 1.20e+03, 8.10e+02],
        [4.40e+00, 7.31e+00, 1.50e+03, 9.40e+02]],

       [[3.26e+00, 6.01e+00, 1.00e+03, 6.80e+02],
        [3.26e+00, 6.01e+00, 1.20e+03, 7.90e+02],
        [3.26e+00, 6.01e+00, 1.50e+03, 9.10e+02]],

       [[3.02e+00, 5.26e+00, 1.00e+03, 6.10e+02],
        [3.02e+00, 5.26e+00, 1.20e+03, 7.00e+02],
        [3.02e+00, 5.26e+00, 1.50e+03, 8.10e+02]],

       [[1.07e+00, 3.86e+00, 1.00e+03, 5.40e+02],
        [1.07e+00, 3.86e+00, 1.20e+03, 6.30e+02],
        [1.07e+00, 3.86e+00, 1.50e+03, 7.30e+02]],

       [[1.52e+00, 2.50e+00, 1.00e+03, 5.00e+02],
        [1.52e+00, 2.50e+00, 1.20e+03, 5.80e+02],
        [1.52e+00, 2.50e+00, 1.50e+03, 6.60e+02]],

       [[4.92e+00, 1.60e-01, 1.00e+03, 5.90e+02],
        [4.92e+00, 1.60e-01, 1.20e+03,

In [8]:
facilities[:,:,2]

array([[1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.],
       [1000., 1200., 1500.]])

Model 2.2.1

In [9]:
Env = gp.Env()
Env.setParam('OutputFlag', 0)
model = gp.Model("2.2.1", env = Env)

#define variables
nFactories = 10
nShops = 200
nPlan = 3
MAX_CAPACITY = facilities[:,:,2].flatten()
x = model.addMVar(shape = (nFactories * nPlan, nShops), lb = 0, ub = 1000, vtype = GRB.INTEGER, name = "x")
# chose_fac = model.addMVar(shape = nFactories, vtype = GRB.BINARY, name = "chose_fac")
chose_plan = model.addMVar(shape = (nFactories * nPlan),vtype = GRB.BINARY, name = "chose_plan")
demand = shops[:,3]
build = facilities[:,:,-1].flatten()
# print(build)
# print(x.shape)
# print(chose_plan.sum(axis=1))
# Define constraints

# Choose 1 plan only for each point
model.addConstrs(chose_plan[i:i+nPlan].sum() <= 1 for i in range(0,nFactories*nPlan,nPlan))

# Production rate is under max capacity
model.addConstr(x.sum(axis = 1) <= MAX_CAPACITY * chose_plan)

# Supply must more than demand for each shop
model.addConstrs(gp.quicksum(x[i:i+nPlan,j].sum() for i in range(0,nFactories*nPlan,nPlan)) >= demand[j] for j in range(nShops))

#define objective
model.setObjective((x * transport_extra).sum() + (chose_plan * build).sum(), GRB.MINIMIZE)

model.update()

model.optimize()

print(model.Runtime)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2395694
Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
0.28600001335144043


Print Model's Objective Value and an optimal solution

In [12]:
if model.status == GRB.OPTIMAL:
    print("One solution found")
    with open("2.2.2.ans","w") as f:
        f.write(f"Total build {(chose_plan.X * build).sum()}\n")
        for i in range(0,nFactories*nPlan,nPlan):
                for k in range(nPlan):
                    if chose_plan[i+k].X > 0:
                        f.write(f'Factory {i//3} is chosen with plan {k} with cost {build[i+k]}\n')
                f.write(f"Total production {x[i:i+nPlan].X.sum()}\n")
        for i in range(0,nFactories*nPlan):
            for j in range(nShops):
                    f.write(f"{abs(x[i][j].X)} ")
            f.write('\n')
        f.write(f"{model.ObjVal}")


else:
    print('No optimal solution found')

One solution found


In [11]:
# if model.status == GRB.OPTIMAL:
#     print("One solution found")
#     for i in range(nFactories):
#         if chose_fac[i].X > 0:
#             print('Factory ', i, ' is chosen')
#             print(x[i].X.sum())
#             for j in range(nShops):
#                 if x[i,j].x > 0:
#                     print(f"Factory {i} sells {x[i][j].X} to shop {j}")
#     print(model.ObjVal)


# else:
#     print('No optimal solution found')